In [23]:
import os
os.getcwd()

'c:\\Users\\146058\\AppData\\Local\\Programs\\Microsoft VS Code'

In [ ]:
# 로그인 (이미 로그인 페이지에 있거나, 필요시 driver.get(LOGIN_URL) 후 실행)
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

w = WebDriverWait(driver, 15)
id = '0000' # 패스워드 입력필요
pw = '0000' # 패스워드 입력필요

# 1) 아이디/비밀번호 입력 + 로그인 클릭
w.until(EC.presence_of_element_located((By.ID, "user_id"))).clear()
w.until(EC.presence_of_element_located((By.ID, "user_id"))).send_keys(id)
w.until(EC.presence_of_element_located((By.ID, "pswd"))).clear()
w.until(EC.presence_of_element_located((By.ID, "pswd"))).send_keys(pw)
w.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.login-btn"))).click()

# 2) 결과 확인: 알림 뜨면 실패로 처리, 아니면 '로그아웃' 등장 or 로그인 폼 사라짐을 성공 기준으로
try:
    # 실패 알림 감지
    WebDriverWait(driver, 3).until(EC.alert_is_present())
    alert = driver.switch_to.alert
    print("로그인 실패(알림):", alert.text.strip())
    alert.accept()
except TimeoutException:
    # 성공 기준 1: 로그아웃 링크가 보임
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//a[contains(.,'로그아웃') or contains(@href,'logout')]"))
        )
        print("로그인 성공")
    except TimeoutException:
        # 성공 기준 2: 로그인 폼이 사라짐
        try:
            WebDriverWait(driver, 5).until(EC.invisibility_of_element_located((By.ID, "user_id")))
            WebDriverWait(driver, 5).until(EC.invisibility_of_element_located((By.ID, "pswd")))
            print("로그인 성공")
        except TimeoutException:
            print("로그인 결과 확인 불가(로그아웃 링크/폼 변화 없음)")


로그인 성공


In [7]:
# -*- coding: utf-8 -*-
# pip install selenium pandas

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, UnexpectedAlertPresentException, NoSuchElementException
import pandas as pd
import re, time

# ===================== CONFIG =====================
URL       = "https://www.csi.go.kr/cmq/qtcSelf/qltRptRslt/qltRptPerConstList.do"
USER_ID   = "jkm2000korea"
USER_PW   = "jeoNG1532!@"

TIMEOUT   = 25
HEADLESS  = True
# ==================================================


# ================= Driver & Utils =================
def build_driver(headless=HEADLESS):
    print("[INIT] Build driver | headless =", headless)
    opts = Options()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1440,2400")
    opts.add_argument("--ignore-certificate-errors")
    opts.add_argument("--allow-insecure-localhost")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.set_capability("acceptInsecureCerts", True)
    d = webdriver.Chrome(options=opts)
    try:
        d.execute_cdp_cmd("Security.setIgnoreCertificateErrors", {"ignore": True})
    except Exception:
        pass
    d.set_page_load_timeout(60)
    print("[INIT] Driver ready")
    return d

def wait_ready(drv):
    WebDriverWait(drv, TIMEOUT).until(
        lambda x: x.execute_script("return document.readyState") == "complete"
    )

def clean_text(el):
    txt = el.get_attribute("innerText") or el.text or ""
    return re.sub(r"\s+", " ", txt).strip()

def accept_any_alert(driver, timeout=5):
    try:
        WebDriverWait(driver, timeout).until(EC.alert_is_present())
        al = driver.switch_to.alert
        print("[ALERT]", (al.text or "").strip())
        al.accept()
        time.sleep(0.2)
        return True
    except TimeoutException:
        return False
# ==================================================


# ============== Inline Login (no redirect) =========
def _find_login_elems_in_context(ctx):
    """현재 컨텍스트(메인/프레임)에서 로그인 입력/버튼 탐색."""
    id_el = pw_el = btn_el = None
    try:
        id_el = ctx.find_element(By.ID, "user_id")
        pw_el = ctx.find_element(By.ID, "pswd")
        # 버튼은 <a class="login-btn"> or href="javascript:go_login('N')"
        btns = ctx.find_elements(By.CSS_SELECTOR, "a.login-btn, a[href*=\"go_login\"]")
        btn_el = btns[0] if btns else None
    except NoSuchElementException:
        pass
    return id_el, pw_el, btn_el

def inline_login(driver, user_id=USER_ID, user_pw=USER_PW):
    """현재 페이지/모달/헤더 등에서 바로 로그인 시도 (별도 페이지 이동 X)."""
    print("[LOGIN] Inline login try on current page...")
    # 경고창 먼저 처리
    accept_any_alert(driver, timeout=3)

    # 1) 메인 DOM에서 먼저 탐색
    id_el, pw_el, btn_el = _find_login_elems_in_context(driver)
    if not (id_el and pw_el and btn_el):
        # 2) 프레임들 안도 검사
        frames = driver.find_elements(By.TAG_NAME, "iframe")
        for idx, f in enumerate(frames):
            try:
                driver.switch_to.frame(f)
                id_el, pw_el, btn_el = _find_login_elems_in_context(driver)
                if id_el and pw_el and btn_el:
                    print(f"[LOGIN] Found login form in iframe #{idx}")
                    break
            except Exception:
                pass
            finally:
                driver.switch_to.default_content()

    # 3) 필요시 '로그인' 오픈 트리거 시도(헤더 버튼 등)
    if not (id_el and pw_el and btn_el):
        candidates = [
            "a[href*='login']", "a.login", "button.login", "a#loginBtn", "a:contains('로그인')"
        ]
        # CSS에 :contains는 표준이 아니라서 텍스트로 찾기 fallback
        clickable = driver.find_elements(By.CSS_SELECTOR, "a, button")
        for c in clickable:
            try:
                txt = (c.text or "").strip()
                if "로그인" in txt:
                    c.click()
                    time.sleep(0.5)
                    accept_any_alert(driver, timeout=2)
                    id_el, pw_el, btn_el = _find_login_elems_in_context(driver)
                    if id_el and pw_el and btn_el:
                        print("[LOGIN] Found login form after clicking a '로그인' trigger")
                        break
            except Exception:
                pass

    if not (id_el and pw_el and btn_el):
        print("[LOGIN] 로그인 폼을 찾지 못했습니다 (현재 페이지).")
        return False

    # 4) 입력 + 클릭
    try:
        id_el.clear(); id_el.send_keys(user_id)
        pw_el.clear(); pw_el.send_keys(user_pw)
        btn_el.click()
        # 알림 있으면 처리
        accept_any_alert(driver, timeout=3)

        # 성공 판정: 로그아웃 보이거나, 로그인 폼이 사라짐
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//a[contains(.,'로그아웃') or contains(@href,'logout')]"))
            )
            print("[LOGIN] 성공 (로그아웃 링크 확인)")
            return True
        except TimeoutException:
            # 폼 사라짐 확인
            try:
                WebDriverWait(driver, 5).until(EC.invisibility_of_element_located((By.ID, "user_id")))
                WebDriverWait(driver, 5).until(EC.invisibility_of_element_located((By.ID, "pswd")))
                print("[LOGIN] 성공 (폼 사라짐)")
                return True
            except TimeoutException:
                print("[LOGIN] 결과 불명확 (계속 진행)")
                return True
    finally:
        # 프레임 전환 원복
        try:
            driver.switch_to.default_content()
        except Exception:
            pass
# ==================================================


# ============== Pagination & Scraping ==============
def get_active_page(drv):
    try:
        a = drv.find_element(By.CSS_SELECTOR, "ul.pagination li.active a")
        title = (a.get_attribute("title") or a.text or "").strip()  # "현재 페이지(1)"
        m = re.search(r"\((\d+)\)", title)
        if m: return int(m.group(1))
        m2 = re.search(r"\d+", a.text or "")
        return int(m2.group(0)) if m2 else None
    except Exception:
        return None

def scrape_main_current_page(drv):
    # 메인 표(id=contents-table) 파싱 + go_detail id 수집
    page_no = get_active_page(drv)
    print(f"[MAIN] Parse page {page_no} ...")
    table = WebDriverWait(drv, TIMEOUT).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "table#contents-table"))
    )
    headers = [clean_text(th) for th in table.find_elements(By.CSS_SELECTOR, "thead th")]
    out = []
    detail_ids = []
    for tr in table.find_elements(By.CSS_SELECTOR, "tbody tr"):
        if not tr.find_elements(By.CSS_SELECTOR, "td"):
            continue
        tds = tr.find_elements(By.CSS_SELECTOR, "td")
        row_vals = [clean_text(td) for td in tds]
        if len(row_vals) < len(headers):
            row_vals += [""] * (len(headers) - len(row_vals))
        elif len(row_vals) > len(headers):
            row_vals = row_vals[:len(headers)]
        row = dict(zip(headers, row_vals))
        onclick = tr.get_attribute("onclick") or ""
        m = re.search(r"go_detail\('([^']+)'\)", onclick)
        go_id = m.group(1) if m else None
        row["상세ID(go_detail)"] = go_id
        out.append(row)
        if go_id:
            detail_ids.append(go_id)

    print(f"[MAIN] Page {page_no}: rows={len(out)} | headers={headers}")
    if detail_ids:
        print(f"[MAIN] Page {page_no}: detail_ids(sample)={detail_ids[:5]} ... (total {len(detail_ids)})")
    return headers, out

def get_pagination_info(drv):
    anchors = drv.find_elements(By.CSS_SELECTOR, "ul.pagination a[href^=\"javascript:goPage('\"]")
    nums, next10 = [], None
    for a in anchors:
        title = (a.get_attribute("title") or "").strip()
        href = a.get_attribute("href") or ""
        m = re.search(r"goPage\('(\d+)'\)", href)
        if not m: continue
        num = int(m.group(1))
        if "다음 10개 페이지" in title:
            next10 = num
        elif "이전 10개 페이지" in title:
            continue
        else:
            nums.append(num)
    nums = sorted(set(nums))
    print(f"[NAV] page_nums={nums} | next10={next10}")
    return nums, next10
# ==================================================


# ======================== MAIN =====================
if __name__ == "__main__":
    driver = build_driver(HEADLESS)

    print("[GET]", URL)
    driver.get(URL)
    # 첫 로드에서 경고(로그인 필요) 뜰 수 있음 → 닫고 같은 페이지에서 로그인
    try:
        wait_ready(driver)
    except UnexpectedAlertPresentException:
        print("[INFO] Unexpected alert during load → accept")
        accept_any_alert(driver, timeout=5)

    # 현재 페이지에서 바로 로그인
    ok = inline_login(driver, USER_ID, USER_PW)
    if not ok:
        print("[WARN] 로그인 시도가 불확실/실패일 수 있음. 계속 시도합니다.")

    # 로그인 후 대상 페이지 새로 열기(동일 URL, 별도 로그인페이지로 이동하지 않음)
    print("[GET-RELOAD]", URL)
    driver.get(URL)
    try:
        wait_ready(driver)
    except UnexpectedAlertPresentException:
        accept_any_alert(driver, timeout=5)
        wait_ready(driver)
    print("[READY] List page after inline login.")

    # --- 메인 표 전 페이지 수집 ---
    main_rows, seen_page = [], set()
    page_group_iter_guard = 0

    _, page_rows = scrape_main_current_page(driver)
    main_rows.extend(page_rows)
    print(f"[ACC] collected(main) = {len(main_rows)}")

    while True:
        page_nums, next10 = get_pagination_info(driver)

        for num in page_nums:
            if num in seen_page:
                print(f"[SKIP] already visited page {num}")
                continue
            print(f"[GO] page {num}")
            driver.execute_script("goPage(arguments[0]);", str(num))
            WebDriverWait(driver, TIMEOUT).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "table#contents-table tbody tr"))
            )
            _, page_rows = scrape_main_current_page(driver)
            main_rows.extend(page_rows)
            seen_page.add(num)
            print(f"[ACC] collected(main) = {len(main_rows)}")
            time.sleep(0.2)

        if next10 and next10 not in seen_page:
            print(f"[GO] next 10-group: page {next10}")
            driver.execute_script("goPage(arguments[0]);", str(next10))
            WebDriverWait(driver, TIMEOUT).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "table#contents-table tbody tr"))
            )
            _, page_rows = scrape_main_current_page(driver)
            main_rows.extend(page_rows)
            seen_page.add(next10)
            print(f"[ACC] collected(main) = {len(main_rows)}")
            page_group_iter_guard += 1
            if page_group_iter_guard > 100:
                print("[GUARD] too many groups → break")
                break
            continue

        print("[DONE] pagination finished.")
        break

    df_main = pd.DataFrame(main_rows)
    print("[RESULT] main rows:", len(df_main), "| columns:", list(df_main.columns))

    # --- 상세 표 수집 (필요 시) ---
    detail_rows_all = []
    for i, row in enumerate(df_main.itertuples(index=False), 1):
        detail_id = getattr(row, "상세ID(go_detail)", None)
        print(f"[DETAIL] ({i}/{len(df_main)}) detail_id =", detail_id)
        if not detail_id:
            print("  -> no detail_id, skip")
            continue
        try:
            driver.execute_script("go_detail(arguments[0]);", str(detail_id))
            wait_ready(driver)

            tbl = WebDriverWait(driver, TIMEOUT).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "table.table.order-column"))
            )
            headers = [clean_text(th) for th in tbl.find_elements(By.CSS_SELECTOR, "thead th")]
            print(f"  -> headers({len(headers)}): {headers}")

            added = 0
            for tr in tbl.find_elements(By.CSS_SELECTOR, "tbody tr"):
                if not tr.find_elements(By.CSS_SELECTOR, "td"):
                    continue
                vals = [clean_text(td) for td in tr.find_elements(By.CSS_SELECTOR, "td")]
                if len(vals) < len(headers):
                    vals += [""] * (len(headers) - len(vals))
                elif len(vals) > len(headers):
                    vals = vals[:len(headers)]
                d = dict(zip(headers, vals))
                links = tr.find_elements(By.CSS_SELECTOR, "td a[href^=\"javascript:go_select('\"]")
                if links:
                    m = re.search(r"go_select\('([^']+)'\)", links[0].get_attribute("href") or "")
                    if m:
                        d["상세행ID(go_select)"] = m.group(1)
                d["상세ID(go_detail)"] = detail_id
                detail_rows_all.append(d)
                added += 1
            print(f"  -> detail rows added: {added} | total detail rows: {len(detail_rows_all)}")
        except Exception as e:
            print("  !! detail error:", repr(e))

        # 목록 복귀
        try:
            driver.back()
            WebDriverWait(driver, TIMEOUT).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "table#contents-table tbody tr"))
            )
            print("  -> back to list OK")
        except Exception as e:
            print("  !! back error:", repr(e))
            break

    driver.quit()

    # 저


[INIT] Build driver | headless = True
[INIT] Driver ready
[GET] https://www.csi.go.kr/cmq/qtcSelf/qltRptRslt/qltRptPerConstList.do
[INFO] Unexpected alert during load → accept
[LOGIN] Inline login try on current page...
[LOGIN] 성공 (로그아웃 링크 확인)
[GET-RELOAD] https://www.csi.go.kr/cmq/qtcSelf/qltRptRslt/qltRptPerConstList.do
[READY] List page after inline login.
[MAIN] Parse page 1 ...
[MAIN] Page 1: rows=10 | headers=['순번', '사업번호', '공사명', '공사기간', '공사구분', '공사종류', '발주자', '시공사', '공공 민간', '총건', '완료건']
[MAIN] Page 1: detail_ids(sample)=['000000034296', '000000034202', '000000033370', '000000032386', '000000031027'] ... (total 10)
[ACC] collected(main) = 10
[NAV] page_nums=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 16] | next10=11
[GO] page 1
[MAIN] Parse page 1 ...
[MAIN] Page 1: rows=10 | headers=['순번', '사업번호', '공사명', '공사기간', '공사구분', '공사종류', '발주자', '시공사', '공공 민간', '총건', '완료건']
[MAIN] Page 1: detail_ids(sample)=['000000034296', '000000034202', '000000033370', '000000032386', '000000031027'] ... (total 1

In [9]:
# 저장
df_detail = pd.DataFrame(detail_rows_all)
df_main.to_csv("메인표_전페이지.csv", index=False, encoding="utf-8-sig")
df_detail.to_csv("상세_템플릿표_전건.csv", index=False, encoding="utf-8-sig")
print("\n[SAVE] 메인표_전페이지.csv :", len(df_main))
print("[SAVE] 상세_템플릿표_전건.csv :", len(df_detail))


[SAVE] 메인표_전페이지.csv : 161
[SAVE] 상세_템플릿표_전건.csv : 0


In [10]:
df_detail

""


In [11]:
df_main

,순번,사업번호,공사명,공사기간,공사구분,공사종류,발주자,시공사,공공 민간,총건,완료건,상세ID(go_detail)
0,1,000000034296,오산세교2지구 M1블록 민간임대 주상복합 신축공사,2025-08-11 ~2029-05-10,건축공사,초고층아파트(16층이상),오산세교아트피에프브이(주),(주)포스코이앤씨,민간,,,000000034296
1,2,000000034202,(포항)6기 코크스 소화탑 내벽 줄눈 보수(공사)_개산계약,2025-09-01 ~2025-11-30,산업설비,제철소 및 석유화학공장등 산업생산시설,주식회사 포스코 김동원,(주)포스코이앤씨,민간,,,000000034202
2,3,000000033370,김해신문1지구 도시개발사업 내 공동주택 2차 신축공사,2025-07-30 ~2028-06-24,건축공사,초고층아파트(16층이상),신영부동산신탁(주),(주)포스코이앤씨,민간,,,000000033370
3,4,000000032386,서리풀 업무복합시설 개발사업 신축공사(북측부지),2025-07-21 ~2029-09-20,건축공사,사무실빌딩,에스비씨피에프브이(주),(주)포스코이앤씨,민간,3,2,000000032386
4,5,000000031027,서리풀 업무복합시설 개발사업 신축공사(남측부지),2025-07-01 ~2030-02-28,건축공사,사무실빌딩,에스비씨피에프브이 주식회사,(주)포스코이앤씨,민간,7,7,000000031027
...,...,...,...,...,...,...,...,...,...,...,...,...
156,146,000000000403,고양 일산 풍동2지구 오피스텔 C-2BL 신축공사,2021-07-23 ~2025-06-30,건축공사,오피스텔,우리자산신탁 주식회사,(주)포스코이앤씨,민간,379,379,000000000403
157,147,000000000379,밀양 삼양식품 2공장 증축공사,2024-04-03 ~2025-05-31,건축공사,"공장, 작업장용건물",삼양식품(주),(주)포스코이앤씨,민간,201,201,000000000379
158,148,000000000356,청주 구룡공원 공동주택 신축공사,2022-01-14 ~2026-01-13,건축공사,초고층아파트(16층이상),주식회사 하나자산신탁,(주)포스코이앤씨,민간,707,695,000000000356
159,149,000000000351,송파 성지아파트 리모델링공사,2022-01-21 ~2025-03-07,건축공사,초고층아파트(16층이상),송파성지아파트리모델링주택조합,(주)포스코이앤씨,민간,149,149,000000000351


In [13]:
# pip install selenium pandas
import re, time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException, UnexpectedAlertPresentException, WebDriverException
)

URL      = "https://www.csi.go.kr/cmq/qtcSelf/qltRptRslt/qltRptPerConstList.do"
USER_ID  = "jkm2000korea"
USER_PW  = "jeoNG1532!@"
TIMEOUT  = 20
HEADLESS = True

# ---------------- Driver helpers ----------------
def build_driver(headless=HEADLESS):
    opts = Options()
    if headless: opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1440,2000")
    opts.add_argument("--ignore-certificate-errors")
    opts.add_argument("--allow-insecure-localhost")
    opts.add_argument("--disable-gpu"); opts.add_argument("--no-sandbox"); opts.add_argument("--disable-dev-shm-usage")
    opts.set_capability("acceptInsecureCerts", True)
    d = webdriver.Chrome(options=opts)
    try:
        d.execute_cdp_cmd("Security.setIgnoreCertificateErrors", {"ignore": True})
    except Exception: pass
    d.set_page_load_timeout(60)
    return d

def wait_ready(drv):
    WebDriverWait(drv, TIMEOUT).until(lambda x: x.execute_script("return document.readyState")=="complete")

def accept_any_alert(drv, timeout=3):
    try:
        WebDriverWait(drv, timeout).until(EC.alert_is_present())
        a = drv.switch_to.alert
        print("[ALERT]", (a.text or "").strip()); a.accept()
        time.sleep(0.2)
    except TimeoutException:
        pass

def inline_login(drv):
    """현재 페이지에서 바로 로그인(별도 페이지 이동 없음)"""
    accept_any_alert(drv, 3)
    # 입력칸 찾아서 입력
    try:
        id_el = WebDriverWait(drv, 5).until(EC.presence_of_element_located((By.ID, "user_id")))
        pw_el = WebDriverWait(drv, 5).until(EC.presence_of_element_located((By.ID, "pswd")))
        btn   = WebDriverWait(drv, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.login-btn, a[href*=\"go_login\"]")))
    except TimeoutException:
        # 헤더/모달 버튼을 눌러 로그인 폼 열어야 하는 경우
        for b in drv.find_elements(By.CSS_SELECTOR, "a,button"):
            if "로그인" in (b.text or ""):
                try: b.click(); time.sleep(0.3); break
                except Exception: pass
        id_el = WebDriverWait(drv, 10).until(EC.presence_of_element_located((By.ID, "user_id")))
        pw_el = WebDriverWait(drv, 10).until(EC.presence_of_element_located((By.ID, "pswd")))
        btn   = WebDriverWait(drv, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.login-btn, a[href*=\"go_login\"]")))
    id_el.clear(); id_el.send_keys(USER_ID)
    pw_el.clear(); pw_el.send_keys(USER_PW)
    btn.click()
    accept_any_alert(drv, 3)
    # 성공 기준: 로그아웃 링크 or 폼 사라짐
    try:
        WebDriverWait(drv, 8).until(
            EC.presence_of_element_located((By.XPATH, "//a[contains(.,'로그아웃') or contains(@href,'logout')]"))
        )
        print("[LOGIN] OK (logout link)")
    except TimeoutException:
        try:
            WebDriverWait(drv, 4).until(EC.invisibility_of_element_located((By.ID, "user_id")))
            print("[LOGIN] OK (form disappeared)")
        except TimeoutException:
            print("[LOGIN] Unclear, proceed")

def driver_alive(drv):
    try:
        drv.execute_script("return 1")
        return True
    except Exception:
        return False

def revive_and_relogin(drv):
    """드라이버 재기동 + 같은 URL 재진입 + 인라인 로그인"""
    try:
        drv.quit()
    except Exception:
        pass
    drv = build_driver(True)
    drv.get(URL)
    try:
        wait_ready(drv)
    except UnexpectedAlertPresentException:
        accept_any_alert(drv, 5)
    inline_login(drv)
    drv.get(URL); wait_ready(drv)
    return drv

# --------------- First-page detail IDs --------------
def collect_detail_ids_on_first_page(drv):
    WebDriverWait(drv, TIMEOUT).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#contents-table tbody tr")))
    rows = drv.find_elements(By.CSS_SELECTOR, "#contents-table tbody tr[onclick*=\"go_detail('\"]")
    ids = []
    for tr in rows:
        oc = tr.get_attribute("onclick") or ""
        m = re.search(r"go_detail\('([^']+)'\)", oc)
        if m: ids.append(m.group(1))
    # 순서 유지 중복 제거
    seen, out = set(), []
    for x in ids:
        if x not in seen:
            out.append(x); seen.add(x)
    print("[IDs 1st page]", out)
    return out

def open_detail_by_id(drv, const_id):
    sel = f"#contents-table tbody tr[onclick*=\"go_detail('{const_id}')\"]"
    tr  = WebDriverWait(drv, TIMEOUT).until(EC.presence_of_element_located((By.CSS_SELECTOR, sel)))
    drv.execute_script("arguments[0].scrollIntoView({block:'center'})", tr); time.sleep(0.05)
    try:
        tr.click()
    except Exception as e:
        print(f"[CLICK fallback] {e} -> call go_detail({const_id})")
        drv.execute_script("window.go_detail && window.go_detail(arguments[0]);", const_id)
    WebDriverWait(drv, TIMEOUT).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "table.table.order-column tbody tr"))
    )

def back_to_list(drv):
    drv.back()
    WebDriverWait(drv, TIMEOUT).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#contents-table tbody tr"))
    )

# ========================= RUN =========================
driver = build_driver(HEADLESS)
try:
    driver.get(URL)
    try:
        wait_ready(driver)
    except UnexpectedAlertPresentException:
        accept_any_alert(driver, 5)
    inline_login(driver)
    driver.get(URL); wait_ready(driver)

    # 1) 첫 페이지에서 상세 ID 수집
    ids = collect_detail_ids_on_first_page(driver)

    # 2) 각 상세로 이동 (드라이버가 죽으면 즉시 복구 후 이어서 진행)
    for i, cid in enumerate(ids, 1):
        print(f"\n== ({i}/{len(ids)}) open detail: {cid}")
        try:
            if not driver_alive(driver):
                print("[DRIVER] dead → revive")
                driver = revive_and_relogin(driver)
            open_detail_by_id(driver, cid)
            print("[OPENED]", cid)
            # TODO: 상세 페이지에서 필요한 데이터 수집
            back_to_list(driver)
            print("[BACK] list")
        except (WebDriverException, TimeoutException) as e:
            # 드라이버 세션 소멸/로컬포트 끊김 등도 여기로 옴
            if not driver_alive(driver) or "Failed to establish a new connection" in str(e) or "Connection refused" in str(e):
                print("[DRIVER] crashed:", e, "→ revive and retry once")
                driver = revive_and_relogin(driver)
                # 재시도 1회
                open_detail_by_id(driver, cid)
                print("[OPENED-RETRY]", cid)
                back_to_list(driver)
            else:
                print("[ERR] detail open failed:", e)
                # 필요시 continue/raise 선택
                continue

finally:
    try:
        driver.quit()
    except Exception:
        pass


[LOGIN] OK (logout link)
[IDs 1st page] ['000000034296', '000000034202', '000000033370', '000000032386', '000000031027', '000000030251', '000000028321', '000000026886', '000000024896', '000000023954']

== (1/10) open detail: 000000034296
[OPENED] 000000034296
[BACK] list

== (2/10) open detail: 000000034202
[OPENED] 000000034202
[ERR] detail open failed: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff7628f3d85+79397]
	GetHandleVerifier [0x0x7ff7628f3de0+79488]
	(No symbol) [0x0x7ff76269c0fa]
	(No symbol) [0x0x7ff7626f2fd6]
	(No symbol) [0x0x7ff7626f328c]
	(No symbol) [0x0x7ff762746537]
	(No symbol) [0x0x7ff76271b1df]
	(No symbol) [0x0x7ff762743344]
	(No symbol) [0x0x7ff76271af73]
	(No symbol) [0x0x7ff7626e41b1]
	(No symbol) [0x0x7ff7626e4f43]
	GetHandleVerifier [0x0x7ff762bbe1ed+3005069]
	GetHandleVerifier [0x0x7ff762bb831d+2980797]
	GetHandleVerifier [0x0x7ff762bd7e0d+3110573]
	GetHandleVerifier [0x0x7ff76290d6de+184190]
	GetHandleVerifier [0x0x7ff76291516f+215567]
	GetHandleVerifie

In [14]:
list

list

In [15]:
df_main

,순번,사업번호,공사명,공사기간,공사구분,공사종류,발주자,시공사,공공 민간,총건,완료건,상세ID(go_detail)
0,1,000000034296,오산세교2지구 M1블록 민간임대 주상복합 신축공사,2025-08-11 ~2029-05-10,건축공사,초고층아파트(16층이상),오산세교아트피에프브이(주),(주)포스코이앤씨,민간,,,000000034296
1,2,000000034202,(포항)6기 코크스 소화탑 내벽 줄눈 보수(공사)_개산계약,2025-09-01 ~2025-11-30,산업설비,제철소 및 석유화학공장등 산업생산시설,주식회사 포스코 김동원,(주)포스코이앤씨,민간,,,000000034202
2,3,000000033370,김해신문1지구 도시개발사업 내 공동주택 2차 신축공사,2025-07-30 ~2028-06-24,건축공사,초고층아파트(16층이상),신영부동산신탁(주),(주)포스코이앤씨,민간,,,000000033370
3,4,000000032386,서리풀 업무복합시설 개발사업 신축공사(북측부지),2025-07-21 ~2029-09-20,건축공사,사무실빌딩,에스비씨피에프브이(주),(주)포스코이앤씨,민간,3,2,000000032386
4,5,000000031027,서리풀 업무복합시설 개발사업 신축공사(남측부지),2025-07-01 ~2030-02-28,건축공사,사무실빌딩,에스비씨피에프브이 주식회사,(주)포스코이앤씨,민간,7,7,000000031027
...,...,...,...,...,...,...,...,...,...,...,...,...
156,146,000000000403,고양 일산 풍동2지구 오피스텔 C-2BL 신축공사,2021-07-23 ~2025-06-30,건축공사,오피스텔,우리자산신탁 주식회사,(주)포스코이앤씨,민간,379,379,000000000403
157,147,000000000379,밀양 삼양식품 2공장 증축공사,2024-04-03 ~2025-05-31,건축공사,"공장, 작업장용건물",삼양식품(주),(주)포스코이앤씨,민간,201,201,000000000379
158,148,000000000356,청주 구룡공원 공동주택 신축공사,2022-01-14 ~2026-01-13,건축공사,초고층아파트(16층이상),주식회사 하나자산신탁,(주)포스코이앤씨,민간,707,695,000000000356
159,149,000000000351,송파 성지아파트 리모델링공사,2022-01-21 ~2025-03-07,건축공사,초고층아파트(16층이상),송파성지아파트리모델링주택조합,(주)포스코이앤씨,민간,149,149,000000000351


In [17]:
# -*- coding: utf-8 -*-
# pip install selenium pandas

"""
목표
- 로그인(현재 페이지에서 직접) → 목록 전 페이지 크롤링(df_main)
- 각 행의 상세(go_detail) 진입 → 상세 테이블 전 건 크롤링(df_detail)
- 상세 DF가 비어 나오던 문제: 클릭 실패/대기 실패/셀렉터 불일치 보완
"""

import re, time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException, UnexpectedAlertPresentException, WebDriverException, NoSuchElementException
)

# =========================== CONFIG ===========================
URL       = "https://www.csi.go.kr/cmq/qtcSelf/qltRptRslt/qltRptPerConstList.do"
USER_ID   = "jkm2000korea"
USER_PW   = "jeoNG1532!@"
TIMEOUT   = 25
HEADLESS  = True
# =============================================================


# ====================== 0) DRIVER & UTILS ====================
def build_driver(headless=HEADLESS) -> webdriver.Chrome:
    print("[INIT] build driver | headless =", headless)
    opts = Options()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1440,2200")
    opts.add_argument("--ignore-certificate-errors")
    opts.add_argument("--allow-insecure-localhost")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.set_capability("acceptInsecureCerts", True)
    d = webdriver.Chrome(options=opts)
    try:
        d.execute_cdp_cmd("Security.setIgnoreCertificateErrors", {"ignore": True})
    except Exception:
        pass
    d.set_page_load_timeout(60)
    return d

def wait_ready(drv: webdriver.Chrome):
    WebDriverWait(drv, TIMEOUT).until(lambda x: x.execute_script("return document.readyState") == "complete")

def accept_any_alert(drv: webdriver.Chrome, timeout=4):
    try:
        WebDriverWait(drv, timeout).until(EC.alert_is_present())
        a = drv.switch_to.alert
        print("[ALERT]", (a.text or "").strip())
        a.accept()
        time.sleep(0.2)
        return True
    except TimeoutException:
        return False

def clean(el) -> str:
    txt = el.get_attribute("innerText") or el.text or ""
    return re.sub(r"\s+", " ", txt).strip()

def driver_alive(drv: webdriver.Chrome) -> bool:
    try:
        drv.execute_script("return 1")
        return True
    except Exception:
        return False
# =============================================================


# ================ 1) INLINE LOGIN (현재 페이지) ================
def _find_login_elems(ctx):
    try:
        id_el = ctx.find_element(By.ID, "user_id")
        pw_el = ctx.find_element(By.ID, "pswd")
        btns  = ctx.find_elements(By.CSS_SELECTOR, "a.login-btn, a[href*=\"go_login\"]")
        btn_el = btns[0] if btns else None
        return id_el, pw_el, btn_el
    except NoSuchElementException:
        return None, None, None

def inline_login(drv: webdriver.Chrome, uid=USER_ID, pw=USER_PW) -> bool:
    """별도 로그인 페이지 이동 없이 현재 화면에서 로그인."""
    print("[LOGIN] inline login start")
    accept_any_alert(drv, 2)

    # 1) 메인 DOM에서 찾기
    id_el, pw_el, btn_el = _find_login_elems(drv)

    # 2) 없으면 iframe 탐색
    if not (id_el and pw_el and btn_el):
        for idx, fr in enumerate(drv.find_elements(By.TAG_NAME, "iframe")):
            try:
                drv.switch_to.frame(fr)
                id_el, pw_el, btn_el = _find_login_elems(drv)
                if id_el and pw_el and btn_el:
                    print(f"[LOGIN] login form found in iframe #{idx}")
                    break
            except Exception:
                pass
            finally:
                drv.switch_to.default_content()

    # 3) 로그인 트리거 버튼(헤더 등) 클릭해서 폼 호출 시도
    if not (id_el and pw_el and btn_el):
        for a in drv.find_elements(By.CSS_SELECTOR, "a,button"):
            if "로그인" in (a.text or ""):
                try:
                    a.click(); time.sleep(0.3); accept_any_alert(drv, 2)
                    id_el, pw_el, btn_el = _find_login_elems(drv)
                    if id_el and pw_el and btn_el:
                        print("[LOGIN] login form opened by trigger click")
                        break
                except Exception:
                    pass

    if not (id_el and pw_el and btn_el):
        print("[LOGIN] login form not found")
        return False

    id_el.clear(); id_el.send_keys(uid)
    pw_el.clear(); pw_el.send_keys(pw)
    btn_el.click()
    accept_any_alert(drv, 3)

    # 성공 판정: 로그아웃 링크 or 폼 사라짐
    try:
        WebDriverWait(drv, 10).until(
            EC.presence_of_element_located((By.XPATH, "//a[contains(.,'로그아웃') or contains(@href,'logout')]"))
        )
        print("[LOGIN] success (logout link)")
        return True
    except TimeoutException:
        try:
            WebDriverWait(drv, 5).until(EC.invisibility_of_element_located((By.ID, "user_id")))
            WebDriverWait(drv, 5).until(EC.invisibility_of_element_located((By.ID, "pswd")))
            print("[LOGIN] success (form disappeared)")
            return True
        except TimeoutException:
            print("[LOGIN] unclear; continue anyway")
            return True
# =============================================================


# =============== 2) 목록 페이지 파싱 & 페이지네이션 ===============
def get_active_page(drv):
    try:
        a = drv.find_element(By.CSS_SELECTOR, "ul.pagination li.active a")
        title = (a.get_attribute("title") or a.text or "").strip()  # "현재 페이지(1)"
        m = re.search(r"\((\d+)\)", title) or re.search(r"(\d+)", title)
        return int(m.group(1)) if m else None
    except Exception:
        return None

def scrape_main_current_page(drv):
    """현재 페이지의 메인 테이블 수집 + go_detail ID."""
    p = get_active_page(drv)
    print(f"[MAIN] parse page {p} ...")
    table = WebDriverWait(drv, TIMEOUT).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "table#contents-table"))
    )
    headers = [clean(th) for th in table.find_elements(By.CSS_SELECTOR, "thead th")]
    out = []
    detail_ids = []
    for tr in table.find_elements(By.CSS_SELECTOR, "tbody tr"):
        if not tr.find_elements(By.CSS_SELECTOR, "td"):  # 빈행 방지
            continue
        vals = [clean(td) for td in tr.find_elements(By.CSS_SELECTOR, "td")]
        if len(vals) < len(headers): vals += [""]*(len(headers)-len(vals))
        if len(vals) > len(headers): vals = vals[:len(headers)]
        row = dict(zip(headers, vals))
        oc = tr.get_attribute("onclick") or ""
        m  = re.search(r"go_detail\('([^']+)'\)", oc)
        row["상세ID(go_detail)"] = m.group(1) if m else None
        out.append(row)
        if m: detail_ids.append(m.group(1))
    print(f"[MAIN] rows={len(out)} | headers={headers}")
    return headers, out, detail_ids

def get_pagination_info(drv):
    anchors = drv.find_elements(By.CSS_SELECTOR, "ul.pagination a[href^=\"javascript:goPage('\"]")
    nums, next10 = [], None
    for a in anchors:
        title = (a.get_attribute("title") or "").strip()
        href = a.get_attribute("href") or ""
        m = re.search(r"goPage\('(\d+)'\)", href)
        if not m: continue
        num = int(m.group(1))
        if "다음 10개 페이지" in title:
            next10 = num
        elif "이전 10개 페이지" in title:
            continue
        else:
            nums.append(num)
    return sorted(set(nums)), next10
# =============================================================


# ================= 3) 상세 테이블 파싱 (핵심 보완) =================
DETAIL_TABLE_SEL = (
    # 사이트마다 클래스 조합이 다를 수 있어 여러 후보를 OR 로
    "table.table.order-column, "
    "table.table-striped.table-bordered.table-hover.order-column, "
    "table.table.table-striped.order-column"
)

def open_detail_by_id(drv, const_id):
    """클릭 → 실패 시 JS로 go_detail(id) 호출 → 상세 로딩 대기"""
    # 항상 새로 행을 찾아 클릭(스테일 회피)
    sel = f"#contents-table tbody tr[onclick*=\"go_detail('{const_id}')\"]"
    try:
        tr = WebDriverWait(drv, TIMEOUT).until(EC.presence_of_element_located((By.CSS_SELECTOR, sel)))
        drv.execute_script("arguments[0].scrollIntoView({block:'center'})", tr)
        time.sleep(0.05)
        try:
            tr.click()
        except Exception as e:
            print(f"[CLICK] failed → JS go_detail({const_id}) | reason:", e)
            drv.execute_script("window.go_detail && window.go_detail(arguments[0]);", const_id)
    except TimeoutException:
        # 목록 요소를 못 잡았으면 JS로 바로 이동
        print("[FIND] list row not found → JS go_detail")
        drv.execute_script("window.go_detail && window.go_detail(arguments[0]);", const_id)

    # 상세 테이블 등장 대기(여러 셀렉터 후보 중 하나라도 로드될 때까지)
    WebDriverWait(drv, TIMEOUT).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, DETAIL_TABLE_SEL))
    )

def parse_detail_tables(drv, const_id, main_row=None):
    """상세 페이지의 한 개(혹은 여러 개) 테이블을 모두 파싱해 레코드로 반환."""
    # 여러 표가 있을 수 있으니 후보 셀렉터로 전부 추출
    tables = drv.find_elements(By.CSS_SELECTOR, DETAIL_TABLE_SEL)
    rows = []
    for idx, tbl in enumerate(tables, 1):
        headers = [clean(th) for th in tbl.find_elements(By.CSS_SELECTOR, "thead th")]
        if not headers:
            # thead 가 없을 수도 있으니 첫 행을 헤더로 가정
            ths = tbl.find_elements(By.CSS_SELECTOR, "tr:first-child th, tr:first-child td")
            headers = [clean(x) for x in ths]
        if not headers:
            continue  # 정말 헤더가 없으면 패스

        for tr in tbl.find_elements(By.CSS_SELECTOR, "tbody tr"):
            tds = tr.find_elements(By.CSS_SELECTOR, "td")
            if not tds:  # 빈 행
                continue
            vals = [clean(td) for td in tds]
            if len(vals) < len(headers): vals += [""]*(len(headers)-len(vals))
            if len(vals) > len(headers): vals = vals[:len(headers)]
            rec = dict(zip(headers, vals))

            # go_select 행 ID 추출(있는 경우)
            link = tr.find_elements(By.CSS_SELECTOR, "a[href^=\"javascript:go_select('\"]")
            if link:
                mm = re.search(r"go_select\('([^']+)'\)", link[0].get_attribute("href") or "")
                if mm:
                    rec["상세행ID(go_select)"] = mm.group(1)

            # 상위 정보도 같이 보존
            rec["상세ID(go_detail)"] = const_id
            if main_row:
                # 메인 행의 주요 키를 같이 적재(필요한 것만 선택)
                for k in ["사업번호", "공사명", "발주자", "시공사", "공사기간", "공사종류", "공사구분"]:
                    if k in main_row:
                        rec[k] = main_row.get(k)
            rows.append(rec)
    return rows

def back_to_list(drv):
    drv.back()
    WebDriverWait(drv, TIMEOUT).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#contents-table tbody tr"))
    )
# =============================================================


# ============================ RUN ============================
if __name__ == "__main__":
    driver = build_driver(HEADLESS)

    # 1) 접속 + 로그인(현재 페이지에서)
    print("[GET]", URL)
    driver.get(URL)
    try:
        wait_ready(driver)
    except UnexpectedAlertPresentException:
        accept_any_alert(driver, 5)

    inline_login(driver)                # 현재 페이지에서 로그인
    driver.get(URL); wait_ready(driver) # 같은 URL 재진입(권한 적용)
    print("[READY] list after login")

    # 2) 목록 전 페이지 수집
    all_main_rows, seen = [], set()
    _, page_rows, _ = scrape_main_current_page(driver)
    all_main_rows.extend(page_rows)

    while True:
        page_nums, next10 = get_pagination_info(driver)

        for n in page_nums:
            if n in seen: 
                continue
            driver.execute_script("goPage(arguments[0]);", str(n))
            WebDriverWait(driver, TIMEOUT).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "#contents-table tbody tr"))
            )
            _, page_rows, _ = scrape_main_current_page(driver)
            all_main_rows.extend(page_rows)
            seen.add(n)
            time.sleep(0.15)

        if next10 and next10 not in seen:
            driver.execute_script("goPage(arguments[0]);", str(next10))
            WebDriverWait(driver, TIMEOUT).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "#contents-table tbody tr"))
            )
            _, page_rows, _ = scrape_main_current_page(driver)
            all_main_rows.extend(page_rows)
            seen.add(next10)
            continue
        break

    df_main = pd.DataFrame(all_main_rows)
    print("[RESULT] df_main shape:", df_main.shape)

    # 3) 상세 수집 (빈 DF 문제 해결: JS 호출 + 멀티 셀렉터 + 강한 대기)
    detail_records = []
    for i, row in enumerate(df_main.itertuples(index=False), 1):
        const_id = getattr(row, "상세ID(go_detail)", None)
        print(f"\n[DETAIL] ({i}/{len(df_main)}) id={const_id}")
        if not const_id:
            print("  - skip (no id)")
            continue

        # 메인 행 dict 준비
        main_row_dict = df_main.iloc[i-1].to_dict()

        try:
            open_detail_by_id(driver, const_id)
            rows = parse_detail_tables(driver, const_id, main_row=main_row_dict)
            print(f"  - parsed rows: {len(rows)}")
            detail_records.extend(rows)
        except Exception as e:
            print("  !! detail error:", repr(e))
        finally:
            try:
                back_to_list(driver)
            except Exception as e:
                print("  !! back error, try reload:", repr(e))
                driver.get(URL); wait_ready(driver)

    driver.quit()

    # 4) DataFrames
    df_detail = pd.DataFrame(detail_records)
    print("[RESULT] df_detail shape:", df_detail.shape)

    # (원하면 저장)
    # df_main.to_csv("메인표_전페이지.csv", index=False, encoding="utf-8-sig")
    # df_detail.to_csv("상세_템플릿표_전건.csv", index=False, encoding="utf-8-sig")


[INIT] build driver | headless = True
[GET] https://www.csi.go.kr/cmq/qtcSelf/qltRptRslt/qltRptPerConstList.do
[LOGIN] inline login start
[LOGIN] success (logout link)
[READY] list after login
[MAIN] parse page 1 ...
[MAIN] rows=10 | headers=['순번', '사업번호', '공사명', '공사기간', '공사구분', '공사종류', '발주자', '시공사', '공공 민간', '총건', '완료건']
[MAIN] parse page 1 ...
[MAIN] rows=10 | headers=['순번', '사업번호', '공사명', '공사기간', '공사구분', '공사종류', '발주자', '시공사', '공공 민간', '총건', '완료건']
[MAIN] parse page 2 ...
[MAIN] rows=10 | headers=['순번', '사업번호', '공사명', '공사기간', '공사구분', '공사종류', '발주자', '시공사', '공공 민간', '총건', '완료건']
[MAIN] parse page 3 ...
[MAIN] rows=10 | headers=['순번', '사업번호', '공사명', '공사기간', '공사구분', '공사종류', '발주자', '시공사', '공공 민간', '총건', '완료건']
[MAIN] parse page 4 ...
[MAIN] rows=10 | headers=['순번', '사업번호', '공사명', '공사기간', '공사구분', '공사종류', '발주자', '시공사', '공공 민간', '총건', '완료건']
[MAIN] parse page 5 ...
[MAIN] rows=10 | headers=['순번', '사업번호', '공사명', '공사기간', '공사구분', '공사종류', '발주자', '시공사', '공공 민간', '총건', '완료건']
[MAIN] parse page 6 .

In [ ]:
# -*- coding: utf-8 -*-
# pip install selenium pandas

import re, time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, UnexpectedAlertPresentException

# ===================== CONFIG =====================
URL_LIST   = "https://www.csi.go.kr/cmq/qtcSelf/qltRptRslt/qltRptPerConstList.do"
URL_DETAIL = "/cmq/qts/sQltRptList.do"  # go_detail 이 submit 하는 경로
USER_ID    = "jkm2000korea"
USER_PW    = "jeoNG1532!@"
TIMEOUT    = 25
HEADLESS   = True
# ==================================================

# ================= DRIVER / UTILS =================
def build_driver(headless=HEADLESS) -> webdriver.Chrome:
    opts = Options()
    if headless: opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1440,2200")
    opts.add_argument("--ignore-certificate-errors")
    opts.add_argument("--allow-insecure-localhost")
    opts.add_argument("--disable-gpu"); opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.set_capability("acceptInsecureCerts", True)
    d = webdriver.Chrome(options=opts)
    try: d.execute_cdp_cmd("Security.setIgnoreCertificateErrors", {"ignore": True})
    except Exception: pass
    d.set_page_load_timeout(60)
    return d

def wait_ready(drv):  # 문서 준비
    WebDriverWait(drv, TIMEOUT).until(lambda x: x.execute_script("return document.readyState")=="complete")

def accept_any_alert(drv, timeout=3):
    try:
        WebDriverWait(drv, timeout).until(EC.alert_is_present())
        a = drv.switch_to.alert
        print("[ALERT]", (a.text or "").strip())
        a.accept()
        time.sleep(0.2)
        return True
    except TimeoutException:
        return False

def clean(el) -> str:
    txt = el.get_attribute("innerText") or el.text or ""
    return re.sub(r"\s+", " ", txt).strip()
# ==================================================

# ===================== LOGIN ======================
def find_login(drv):
    try:
        id_el = drv.find_element(By.ID, "user_id")
        pw_el = drv.find_element(By.ID, "pswd")
        btn   = drv.find_element(By.CSS_SELECTOR, "a.login-btn, a[href*=\"go_login\"]")
        return id_el, pw_el, btn
    except Exception:
        # iframe 내부 탐색
        for fr in drv.find_elements(By.TAG_NAME, "iframe"):
            try:
                drv.switch_to.frame(fr)
                id_el = drv.find_element(By.ID, "user_id")
                pw_el = drv.find_element(By.ID, "pswd")
                btn   = drv.find_element(By.CSS_SELECTOR, "a.login-btn, a[href*=\"go_login\"]")
                return id_el, pw_el, btn
            except Exception:
                drv.switch_to.default_content()
                continue
        drv.switch_to.default_content()
        return None, None, None

def inline_login(drv, uid=USER_ID, pw=USER_PW):
    print("[LOGIN] try inline")
    id_el, pw_el, btn = find_login(drv)
    if not (id_el and pw_el and btn):
        print("[LOGIN] form not visible → already logged in? continue")
        return True
    id_el.clear(); id_el.send_keys(uid)
    pw_el.clear(); pw_el.send_keys(pw)
    btn.click()
    accept_any_alert(drv, 4)
    # 로그아웃 버튼 존재로 확인
    try:
        WebDriverWait(drv, 10).until(
            EC.presence_of_element_located((By.XPATH, "//a[contains(.,'로그아웃') or contains(@href,'logout')]"))
        )
        print("[LOGIN] success")
        return True
    except TimeoutException:
        print("[LOGIN] proceed (cannot verify but no error)")
        return True
# ==================================================

# ============== 목록 페이지 수집 ==================
def scrape_main_current_page(drv):
    """현재 페이지의 메인 테이블 수집 + go_detail ID"""
    table = WebDriverWait(drv, TIMEOUT).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "table#contents-table"))
    )
    headers = [clean(th) for th in table.find_elements(By.CSS_SELECTOR, "thead th")]
    out, ids = [], []
    for tr in table.find_elements(By.CSS_SELECTOR, "tbody tr"):
        tds = tr.find_elements(By.CSS_SELECTOR, "td")
        if not tds: continue
        vals = [clean(td) for td in tds]
        if len(vals) < len(headers): vals += [""]*(len(headers)-len(vals))
        if len(vals) > len(headers): vals = vals[:len(headers)]
        row = dict(zip(headers, vals))
        oc = tr.get_attribute("onclick") or ""
        m = re.search(r"go_detail\('([^']+)'\)", oc)
        row["상세ID(go_detail)"] = m.group(1) if m else None
        out.append(row)
        if m: ids.append(m.group(1))
    return headers, out, ids

def get_pagination_info(drv):
    anchors = drv.find_elements(By.CSS_SELECTOR, "ul.pagination a[href^=\"javascript:goPage('\"]")
    nums, next10 = [], None
    for a in anchors:
        title = (a.get_attribute("title") or "").strip()
        href  = a.get_attribute("href") or ""
        m = re.search(r"goPage\('(\d+)'\)", href)
        if not m: continue
        num = int(m.group(1))
        if "다음 10개 페이지" in title: next10 = num
        elif "이전 10개 페이지" in title: continue
        else: nums.append(num)
    return sorted(set(nums)), next10

def collect_all_main(drv):
    print("[MAIN] collect all pages...")
    all_rows, seen = [], set()
    _, rows, _ = scrape_main_current_page(drv)
    all_rows += rows
    while True:
        page_nums, next10 = get_pagination_info(drv)
        for n in page_nums:
            if n in seen: continue
            drv.execute_script("goPage(arguments[0]);", str(n))
            WebDriverWait(drv, TIMEOUT).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "#contents-table tbody tr"))
            )
            _, rows, _ = scrape_main_current_page(drv)
            all_rows += rows
            seen.add(n)
            time.sleep(0.1)
        if next10 and next10 not in seen:
            drv.execute_script("goPage(arguments[0]);", str(next10))
            WebDriverWait(drv, TIMEOUT).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "#contents-table tbody tr"))
            )
            _, rows, _ = scrape_main_current_page(drv)
            all_rows += rows
            seen.add(next10)
            continue
        break
    df_main = pd.DataFrame(all_rows)
    print("[MAIN] done →", df_main.shape)
    return df_main
# ==================================================

# ============== 상세 페이지(POST submit) ==============
XPATH_DETAIL_TABLE = (
    # summary 기반(페이지 소스에 존재)
    "//table[contains(@summary,'사업명') and contains(@summary,'재료명')]"
    " | //table[caption[contains(normalize-space(.),'템플릿 리스트')]]"
    " | //table[contains(@class,'order-column') and .//th[normalize-space()='사업명'] and .//th[normalize-space()='재료명']]"
)

def open_detail_via_post(drv, biz_no):
    """목록 페이지로 이동 → form#main에 biz_no 세팅 → /qts/sQltRptList.do 로 submit"""
    drv.get(URL_LIST)
    wait_ready(drv)
    js = """
    var f = document.getElementById('main');
    if(!f || !f.biz_no){ return 'noform'; }
    f.biz_no.value = arguments[0];
    f.action = arguments[1];
    f.target = '_self';
    f.submit();
    return 'submitted';
    """
    res = drv.execute_script(js, biz_no, URL_DETAIL)
    print(f"[DETAIL] submit biz_no={biz_no} → {res}")
    accept_any_alert(drv, 3)
    # 상세 테이블 등장 대기
    WebDriverWait(drv, TIMEOUT).until(
        EC.presence_of_element_located((By.XPATH, XPATH_DETAIL_TABLE))
    )

def parse_detail(drv, biz_no, main_row=None):
    """상세 페이지의 템플릿 리스트 테이블 전 행 파싱"""
    tables = drv.find_elements(By.XPATH, XPATH_DETAIL_TABLE)
    records = []
    for tbl in tables:
        # 헤더 우선 수집
        ths = tbl.find_elements(By.CSS_SELECTOR, "thead th")
        headers = [clean(th) for th in ths] if ths else []
        if not headers:
            # 첫 행을 헤더로 가정
            fh = tbl.find_elements(By.CSS_SELECTOR, "tr:first-child th, tr:first-child td")
            headers = [clean(x) for x in fh]
        if not headers:
            continue
        for tr in tbl.find_elements(By.CSS_SELECTOR, "tbody tr"):
            tds = tr.find_elements(By.CSS_SELECTOR, "td")
            if not tds: continue
            vals = [clean(td) for td in tds]
            if len(vals) < len(headers): vals += [""]*(len(headers)-len(vals))
            if len(vals) > len(headers): vals = vals[:len(headers)]
            rec = dict(zip(headers, vals))
            # go_select 행 ID 추출(있는 경우)
            a = tr.find_elements(By.CSS_SELECTOR, "a[href^=\"javascript:go_select('\"]")
            if a:
                m = re.search(r"go_select\('([^']+)'\)", a[0].get_attribute("href") or "")
                if m: rec["상세행ID(go_select)"] = m.group(1)
            rec["상세ID(go_detail)"] = biz_no
            if main_row is not None:
                for k in ["사업번호","공사명","발주자","시공사","공사기간","공사종류","공사구분","공공 민간","총건","완료건"]:
                    if k in main_row: rec[k] = main_row.get(k)
            records.append(rec)
    return records
# ==================================================

# ============================ RUN ============================
if __name__ == "__main__":
    driver = build_driver(HEADLESS)

    # 1) 목록 페이지 접속 & 로그인
    print("[GET]", URL_LIST)
    driver.get(URL_LIST)
    try:
        wait_ready(driver)
    except UnexpectedAlertPresentException:
        accept_any_alert(driver, 5)
    inline_login(driver)
    # 권한 반영 위해 같은 URL 재호출
    driver.get(URL_LIST); wait_ready(driver)

    # 2) 목록 전 페이지 수집
    df_main = collect_all_main(driver)

    # 3) 상세 수집: **폼 POST 제출 방식**으로 전환
    detail_records = []
    unique_ids = [x for x in df_main["상세ID(go_detail)"].dropna().unique().tolist()]
    print("[DETAIL] total biz_no to fetch:", len(unique_ids))

    for i, biz_no in enumerate(unique_ids, 1):
        print(f"\n[DETAIL] ({i}/{len(unique_ids)}) biz_no={biz_no}")
        try:
            open_detail_via_post(driver, biz_no)
            # 디버깅용: 현재 URL/제목 출력
            try:
                print("   url:", driver.current_url, "| title:", driver.title)
            except Exception:
                pass
            main_row_dict = None
            # 같은 biz_no 행 하나만 매칭(있을 때)
            hit = df_main.loc[df_main["상세ID(go_detail)"]==biz_no]
            if not hit.empty:
                main_row_dict = hit.iloc[0].to_dict()
            recs = parse_detail(driver, biz_no, main_row=main_row_dict)
            print("   parsed rows:", len(recs))
            if not recs:
                # 페이지 저장(문제 상황 확인용)
                with open(f"detail_empty_{biz_no}.html", "w", encoding="utf-8") as f:
                    f.write(driver.page_source)
                print(f"   !! no rows → saved: detail_empty_{biz_no}.html")
            detail_records.extend(recs)
        except Exception as e:
            print("   !! error:", repr(e))
            # 실패 시에도 다음 건을 위해 계속 진행

    driver.quit()

    # 4) 결과 DF
    df_detail = pd.DataFrame(detail_records)
    print("\n[RESULT] df_main:", df_main.shape, "| columns:", list(df_main.columns))
    print("[RESULT] df_detail:", df_detail.shape, "| columns:", list(df_detail.columns))

# 저장
df_main.to_csv("Quality_test_site_list.csv", index=False, encoding="utf-8-sig")
df_detail.to_csv("Quality_test_atsite.csv", index=False, encoding="utf-8-sig")


[GET] https://www.csi.go.kr/cmq/qtcSelf/qltRptRslt/qltRptPerConstList.do
[LOGIN] try inline
[LOGIN] success
[MAIN] collect all pages...
[MAIN] done → (161, 12)
[DETAIL] total biz_no to fetch: 151

[DETAIL] (1/151) biz_no=000000034296
[DETAIL] submit biz_no=000000034296 → submitted
   url: https://www.csi.go.kr/cmq/qts/sQltRptList.do | title: 품질시험·검사 > 자체 품질시험·검사 > 자체 품질검사 목록
   parsed rows: 1

[DETAIL] (2/151) biz_no=000000034202
[DETAIL] submit biz_no=000000034202 → submitted
   url: https://www.csi.go.kr/cmq/qts/sQltRptList.do | title: 품질시험·검사 > 자체 품질시험·검사 > 자체 품질검사 목록
   parsed rows: 1

[DETAIL] (3/151) biz_no=000000033370
[DETAIL] submit biz_no=000000033370 → submitted
   url: https://www.csi.go.kr/cmq/qts/sQltRptList.do | title: 품질시험·검사 > 자체 품질시험·검사 > 자체 품질검사 목록
   parsed rows: 1

[DETAIL] (4/151) biz_no=000000032386
[DETAIL] submit biz_no=000000032386 → submitted
   url: https://www.csi.go.kr/cmq/qts/sQltRptList.do | title: 품질시험·검사 > 자체 품질시험·검사 > 자체 품질검사 목록
   parsed rows: 3

[DE

In [20]:
df_main

,순번,사업번호,공사명,공사기간,공사구분,공사종류,발주자,시공사,공공 민간,총건,완료건,상세ID(go_detail)
0,1,000000034296,오산세교2지구 M1블록 민간임대 주상복합 신축공사,2025-08-11 ~2029-05-10,건축공사,초고층아파트(16층이상),오산세교아트피에프브이(주),(주)포스코이앤씨,민간,,,000000034296
1,2,000000034202,(포항)6기 코크스 소화탑 내벽 줄눈 보수(공사)_개산계약,2025-09-01 ~2025-11-30,산업설비,제철소 및 석유화학공장등 산업생산시설,주식회사 포스코 김동원,(주)포스코이앤씨,민간,,,000000034202
2,3,000000033370,김해신문1지구 도시개발사업 내 공동주택 2차 신축공사,2025-07-30 ~2028-06-24,건축공사,초고층아파트(16층이상),신영부동산신탁(주),(주)포스코이앤씨,민간,,,000000033370
3,4,000000032386,서리풀 업무복합시설 개발사업 신축공사(북측부지),2025-07-21 ~2029-09-20,건축공사,사무실빌딩,에스비씨피에프브이(주),(주)포스코이앤씨,민간,3,2,000000032386
4,5,000000031027,서리풀 업무복합시설 개발사업 신축공사(남측부지),2025-07-01 ~2030-02-28,건축공사,사무실빌딩,에스비씨피에프브이 주식회사,(주)포스코이앤씨,민간,7,7,000000031027
...,...,...,...,...,...,...,...,...,...,...,...,...
156,146,000000000403,고양 일산 풍동2지구 오피스텔 C-2BL 신축공사,2021-07-23 ~2025-06-30,건축공사,오피스텔,우리자산신탁 주식회사,(주)포스코이앤씨,민간,379,379,000000000403
157,147,000000000379,밀양 삼양식품 2공장 증축공사,2024-04-03 ~2025-05-31,건축공사,"공장, 작업장용건물",삼양식품(주),(주)포스코이앤씨,민간,201,201,000000000379
158,148,000000000356,청주 구룡공원 공동주택 신축공사,2022-01-14 ~2026-01-13,건축공사,초고층아파트(16층이상),주식회사 하나자산신탁,(주)포스코이앤씨,민간,707,695,000000000356
159,149,000000000351,송파 성지아파트 리모델링공사,2022-01-21 ~2025-03-07,건축공사,초고층아파트(16층이상),송파성지아파트리모델링주택조합,(주)포스코이앤씨,민간,149,149,000000000351


In [21]:
df_detail

,NO,구분번호,사업명,재료명,시험·검사 장소,시험내용,등록자,등록일,시험자,검토자,...,공사명,발주자,시공사,공사기간,공사종류,공사구분,공공 민간,총건,완료건,상세행ID(go_select)
0,등록된 자료가 없습니다.,,,,,,,,,,...,오산세교2지구 M1블록 민간임대 주상복합 신축공사,오산세교아트피에프브이(주),(주)포스코이앤씨,2025-08-11 ~2029-05-10,초고층아파트(16층이상),건축공사,민간,,,NaN
1,등록된 자료가 없습니다.,,,,,,,,,,...,(포항)6기 코크스 소화탑 내벽 줄눈 보수(공사)_개산계약,주식회사 포스코 김동원,(주)포스코이앤씨,2025-09-01 ~2025-11-30,제철소 및 석유화학공장등 산업생산시설,산업설비,민간,,,NaN
2,등록된 자료가 없습니다.,,,,,,,,,,...,김해신문1지구 도시개발사업 내 공동주택 2차 신축공사,신영부동산신탁(주),(주)포스코이앤씨,2025-07-30 ~2028-06-24,초고층아파트(16층이상),건축공사,민간,,,NaN
3,1,SQ-20250906-00302-00,서리풀 업무복합시설 개발사업 신축공사(북측부지),그라우트,현장내,그라우트 팽창재 : 1건,류성열,2025-09-06,류성열,이창환,...,서리풀 업무복합시설 개발사업 신축공사(북측부지),에스비씨피에프브이(주),(주)포스코이앤씨,2025-07-21 ~2029-09-20,사무실빌딩,건축공사,민간,3,2,1417623
4,2,SQ-20250903-00038-00,서리풀 업무복합시설 개발사업 신축공사(북측부지),굳지않은콘크리트(25-21-120)_10002,가설도로/가설도로 gate1~gate2,굳지 아니한 콘크리트(레미콘포함) : 10건,류성열,2025-09-03,류성열,이창환,...,서리풀 업무복합시설 개발사업 신축공사(북측부지),에스비씨피에프브이(주),(주)포스코이앤씨,2025-07-21 ~2029-09-20,사무실빌딩,건축공사,민간,3,2,1404676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,6,SQ-20250902-03231-00,용인 에코타운 조성 민간투자사업,굳지않은콘크리트(25-24-150),"[20019]혐기성소화조 휀룸,계단실 벽체 및 슬라브",굳지 아니한 콘크리트(레미콘포함) : 5건,최윤성,2025-08-29,최윤성,유운찬,...,용인 에코타운 조성 민간투자사업,휴먼에코랜드 주식회사,(주)포스코이앤씨,2022-03-11 ~2026-03-10,하수도,토목공사,민간,474,474,1389205
1378,7,SQ-20250902-03146-00,용인 에코타운 조성 민간투자사업,굳지않은콘크리트(25-30-150)_10334,"지하구내도로,악취제거설비/17번 광덕트 및 트레이슬라브, 상부슬라브 개구부 1개소",굳지 아니한 콘크리트(레미콘포함) : 5건,최윤성,2025-08-29,최윤성,유운찬,...,용인 에코타운 조성 민간투자사업,휴먼에코랜드 주식회사,(주)포스코이앤씨,2022-03-11 ~2026-03-10,하수도,토목공사,민간,474,474,1388223
1379,8,SQ-20250828-00696-00,용인 에코타운 조성 민간투자사업,굳지않은콘크리트(25-18-150)_10333,지하구내도로/11번 버림,굳지 아니한 콘크리트(레미콘포함) : 5건,최윤성,2025-08-26,최윤성,유운찬,...,용인 에코타운 조성 민간투자사업,휴먼에코랜드 주식회사,(주)포스코이앤씨,2022-03-11 ~2026-03-10,하수도,토목공사,민간,474,474,1373602
1380,9,SQ-20250828-00686-00,용인 에코타운 조성 민간투자사업,굳지않은콘크리트(25-21-150)_10332,폐수처리시설/B1F 기계패드,굳지 아니한 콘크리트(레미콘포함) : 5건,최윤성,2025-08-26,최윤성,유운찬,...,용인 에코타운 조성 민간투자사업,휴먼에코랜드 주식회사,(주)포스코이앤씨,2022-03-11 ~2026-03-10,하수도,토목공사,민간,474,474,1373600


In [ ]:
# 저장
df_main.to_csv("Quality_test_site_list.csv", index=False, encoding="utf-8-sig")
df_detail.to_csv("Quality_test_atsite.csv", index=False, encoding="utf-8-sig")